# TP en Traitement Automatique du Langage Naturel : Classification des Sentiments sur des Critiques de Films

## Objectif
Développer un système de classification des sentiments en utilisant des critiques de films. Vous utiliserez l'ensemble de données IMDb et appliquerez un modèle **K-Nearest Neighbors (KNN)** pour classer les critiques en catégories positives ou négatives.

---

## Questions
1. Comment la réduction du nombre de caractéristiques (`max_features`) influence-t-elle la performance du modèle ?
2. Quel est l'impact du choix du **nombre de voisins** dans KNN sur les résultats ?
3. Comparez les performances du modèle KNN avec un autre classificateur (par exemple, [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) ou [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)). Lequel fonctionne le mieux et pourquoi ?
4. Le prétraitement améliore-t-il la classification ? Justifiez votre réponse avec des résultats expérimentaux.

---

## Ressources Utiles
- [Ensemble de données IMDb](https://huggingface.co/datasets/imdb)
- [Documentation Scikit-learn](https://scikit-learn.org/stable/)

In [13]:
import copy
!pip install -q -U datasets scikit-learn spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [22]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import copy

# On split plusieur fois le dataset afin de réduire le temps de calcule
dataset = load_dataset("imdb", split="train")
original_dataset = dataset.train_test_split(stratify_by_column="label", test_size=0.5, seed=42)
clean_dataset = copy.deepcopy(original_dataset)

In [23]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [24]:
def clean_sentence_batch(texts):
    # Utilise le pipeline NLP pour traiter un lot de textes (texts) en désactivant les composants inutiles (analyseur et reconnaissance d'entités nommées négligés pour améliorer la performance)
    docs = nlp.pipe(texts, disable=["parser", "ner"])

    # Initialisation d'une liste pour stocker les résultats des textes nettoyés
    result = []

    for doc in docs:
        # Parcours de chaque document renvoyé par le pipeline NLP
        clean_doc = []  # Initialisation d'une liste pour le document nettoyé
        for token in doc:
            # Pour chaque token du document :
            # - Le token est ignoré s'il est un mot vide (stopword) ou un signe de ponctuation
            if not token.is_stop and not token.is_punct:
                # Ajoute le lemme (forme de base du mot) au document nettoyé
                clean_doc.append(token.lemma_)
        # Joint les lemmes en une chaîne et ajoute le résultat à la liste finale
        result.append(" ".join(clean_doc))

    # Retourne la liste contenant les phrases nettoyées
    return result

In [25]:
clean_sentence_batch(["This is a sentence",
                      "This is another sentence",
                      "This is a third sentence with stopwords, punctuation, and numbers."])

['sentence', 'sentence', 'sentence stopword punctuation number']

In [26]:
# Nettoie les phrases dans le jeu de données d'entraînement en utilisant la fonction `clean_sentence_batch`
clean_dataset['train'] = clean_dataset['train'].map(
        lambda x: {"text": clean_sentence_batch(x["text"])},
        # Applique le nettoyage sur la colonne "text" en mode batch
        batched=True  # Permet le traitement par lots pour améliorer les performances
        )

# Nettoie les phrases dans le jeu de données de test de la même manière
clean_dataset['test'] = clean_dataset['test'].map(
        lambda x: {"text": clean_sentence_batch(x["text"])},  # Applique la même transformation au jeu de test
        batched=True  # Assure également un traitement par lots ici
        )

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [28]:
original_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
})

In [30]:
print(original_dataset['train'][0]['text'])
print()
print(clean_dataset['train'][0]['text'])

I really enjoyed the performances of the main cast. Emma Lung is courageous and interesting. The director has developed performances where the characters are not one dimensional. A complex story with the changing between eras. Also appreciated the underlying story of the unions losing power and the effect of a large employer closing on a small town. I do not agree with the comment that the older man has to be attractive. There have be many relationships with older men and younger women - without the male being good looking. Depth of character can be appealing to the not so shallow. The film has a good look and the cinematography is also good.

enjoy performance main cast Emma Lung courageous interesting director develop performance character dimensional complex story changing era appreciate underlie story union lose power effect large employer close small town agree comment old man attractive relationship old man young woman male good look depth character appeal shallow film good look 

In [31]:
# Constantes
TFIDF_MAX_FEATURES = 1000  # Nombre maximal de caractéristiques à extraire en mode TF-IDF
KNN_NEIGHBORS = 100  # Nombre de voisins à considérer dans l'algorithme KNN


def vectorize_text(train_texts, test_texts, max_features):
    """
    Vectorise les textes d'entraînement et de test à l'aide de TF-IDF.
    Chaque document est représenté par un vecteur de caractéristiques numériques.

    Paramètres :
    - train_texts (list): Liste des textes de l'ensemble d'entraînement.
    - test_texts (list): Liste des textes de l'ensemble de test.
    - max_features (int): Nombre maximal de caractéristiques considérées par le vectoriseur.

    Retourne :
    - X_train (sparse matrix): Matrice de caractéristiques pour les données d'entraînement.
    - X_test (sparse matrix): Matrice de caractéristiques pour les données de test.
    - vectorizer (TfidfVectorizer): Instance du vectoriseur TF-IDF entraînée pour réutilisation.
    """
    vectorizer = TfidfVectorizer(
            max_features=max_features)  # Initialisation du vectoriseur avec un seuil max de caractéristiques
    X_train = vectorizer.fit_transform(train_texts)  # Calcul et transformation des textes d'entraînement
    X_test = vectorizer.transform(test_texts)  # Transformation des textes de test avec le même vectoriseur
    return X_train, X_test, vectorizer


def train_and_evaluate_knn(X_train, y_train, X_test, y_test, n_neighbors):
    """
    Entraîne et évalue un modèle KNN pour la classification des textes.

    Paramètres :
    - X_train (sparse matrix): Matrice des caractéristiques pour l'entraînement.
    - y_train (array): Étiquettes associées aux données d'entraînement.
    - X_test (sparse matrix): Matrice des caractéristiques pour les données de test.
    - y_test (array): Étiquettes associées aux données de test.
    - n_neighbors (int): Nombre de voisins pris en compte dans l'algorithme KNN.

    Comportement :
    - Entraîne le modèle sur les données d'entraînement.
    - Effectue des prédictions sur les données de test.
    - Affiche un rapport de classification basé sur les prédictions.
    """
    knn = KNeighborsClassifier(
            n_neighbors=n_neighbors)  # Initialisation du classificateur KNN avec un certain nombre de voisins
    knn.fit(X_train, y_train)  # Entraînement du modèle sur les données d'entrée
    y_pred = knn.predict(X_test)  # Prédictions sur les données de test
    print(classification_report(y_test, y_pred))  # Affichage des métriques de classification (précision, rappel, etc.)

In [32]:
# Flux principal du script
# Vectorisation des textes d'entraînement et de test
X_train, X_test, vectorizer = vectorize_text(original_dataset['train']['text'], original_dataset['test']['text'], TFIDF_MAX_FEATURES)

# Séparation des étiquettes pour les ensembles d'entraînement et de test
y_train, y_test = original_dataset['train']['label'], original_dataset['test']['label']

# Formation et évaluation du modèle KNN avec les données vectorisées
train_and_evaluate_knn(X_train, y_train, X_test, y_test, KNN_NEIGHBORS)

              precision    recall  f1-score   support

           0       0.79      0.62      0.70      6250
           1       0.69      0.83      0.75      6250

    accuracy                           0.73     12500
   macro avg       0.74      0.73      0.73     12500
weighted avg       0.74      0.73      0.73     12500



In [33]:
# Flux principal du script
# Vectorisation des textes d'entraînement et de test
X_train, X_test, vectorizer = vectorize_text(clean_dataset['train']['text'], clean_dataset['test']['text'], TFIDF_MAX_FEATURES)

# Séparation des étiquettes pour les ensembles d'entraînement et de test
y_train, y_test = clean_dataset['train']['label'], clean_dataset['test']['label']

# Formation et évaluation du modèle KNN avec les données vectorisées
train_and_evaluate_knn(X_train, y_train, X_test, y_test, KNN_NEIGHBORS)

              precision    recall  f1-score   support

           0       0.73      0.83      0.77      6250
           1       0.80      0.69      0.74      6250

    accuracy                           0.76     12500
   macro avg       0.76      0.76      0.76     12500
weighted avg       0.76      0.76      0.76     12500

